# 01 - QLoRA Fine-Tuning Lab

## Overview

This is a self-guided lab for fine-tuning a 7B-parameter language model on
legal data using QLoRA (Quantized Low-Rank Adaptation). You will work through
five checkpoints at your own pace. Each checkpoint builds on the previous one,
but you can stop at any point and resume later.

**What is QLoRA?** QLoRA combines two techniques:
1. **4-bit quantization** -- compress the model weights from 16-bit to 4-bit,
   reducing GPU memory by roughly 4x.
2. **LoRA adapters** -- instead of updating all model parameters, add small
   trainable matrices to specific layers. These adapters typically represent
   less than 1% of the total parameter count.

The result: you can fine-tune a 7B model on a single consumer GPU (16 GB VRAM)
that would otherwise require 4x A100s for full fine-tuning.

**Prerequisites:** Modules 06 (SFT) and 07 (RLHF/alignment). Module 08
(evaluation) is recommended for Checkpoint 4.

---

**GPU REQUIREMENT**: This notebook requires an NVIDIA GPU with at least
16 GB VRAM (e.g., RTX 4090, A100, or equivalent cloud GPU). It will not
run on CPU-only machines or Apple Silicon.

---

## Checkpoint 1: Setup and Quantization

### Why quantize?

A 7B-parameter model in float16 requires approximately 14 GB of GPU memory
just for the weights (7 billion parameters x 2 bytes each). That leaves
almost no room for optimizer states, gradients, or activations during training.

4-bit quantization reduces the weight memory to roughly 3.5 GB (7B x 0.5 bytes),
freeing up space for the training overhead. The key insight behind QLoRA is that
you can quantize the base model for memory efficiency while keeping the small
LoRA adapter weights in full precision for training stability.

### Quantization types: NF4 vs FP4

Two common 4-bit formats exist:

- **FP4 (4-bit floating point)**: A uniform quantization grid. Works well for
  uniformly distributed values, but neural network weights are not uniform --
  they tend to follow a normal distribution centered around zero.

- **NF4 (4-bit NormalFloat)**: Designed specifically for normally-distributed
  weights. The quantization levels are spaced according to a normal distribution,
  placing more precision near zero where most weights cluster. This yields better
  accuracy than FP4 for transformer models.

We use NF4 because pretrained transformer weights are approximately normally
distributed, making NF4 the better choice for preserving model quality.

### Double quantization

Standard quantization stores a scaling factor for each block of weights
(typically 64 weights per block). These scaling factors themselves consume
memory. Double quantization applies a second round of quantization to the
scaling factors, saving an additional ~0.4 GB for a 7B model.

In [ ]:
import torch

if not torch.cuda.is_available():
    raise RuntimeError(
        "This notebook requires an NVIDIA GPU with CUDA support. "
        "No CUDA device was detected. If you are on a cloud instance, "
        "make sure you selected a GPU runtime."
    )

device = torch.device("cuda")
gpu_name = torch.cuda.get_device_name(0)
gpu_mem_gb = torch.cuda.get_device_properties(0).total_mem / 1e9

print(f"GPU: {gpu_name}")
print(f"Total VRAM: {gpu_mem_gb:.1f} GB")
print(f"CUDA version: {torch.version.cuda}")
print(f"PyTorch version: {torch.__version__}")

if gpu_mem_gb < 15:
    print(
        f"\nWARNING: Your GPU has {gpu_mem_gb:.1f} GB VRAM. "
        f"This lab recommends at least 16 GB. You may encounter "
        f"out-of-memory errors during training."
    )

In [ ]:
# Install required packages (uncomment if needed)
# !pip install transformers peft bitsandbytes trl datasets accelerate

In [ ]:
import json
import os
from pathlib import Path

import matplotlib.pyplot as plt
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
def print_gpu_memory(label=""):
    """Print current GPU memory usage."""
    allocated = torch.cuda.memory_allocated() / 1e9
    reserved = torch.cuda.memory_reserved() / 1e9
    total = torch.cuda.get_device_properties(0).total_mem / 1e9
    print(f"[{label}] GPU Memory: {allocated:.2f} GB allocated, "
          f"{reserved:.2f} GB reserved, {total:.1f} GB total")


print_gpu_memory("Before loading")

In [ ]:
# Choose your model -- Mistral-7B-v0.1 is a strong open base model.
# You can also use "meta-llama/Llama-2-7b-hf" (requires Llama 2 license acceptance).
model_name = "mistralai/Mistral-7B-v0.1"

# Configure 4-bit quantization with NF4
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"Loading {model_name} with 4-bit NF4 quantization...")
print("This may take a few minutes on first download.")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

print(f"\nModel loaded successfully.")
print_gpu_memory("After loading quantized model")

In [ ]:
# Inspect the quantized model structure
print(f"Model type: {type(model).__name__}")
print(f"Model dtype: {model.dtype}")
print(f"\nModel architecture (first few layers):")

# Show the first transformer layer to see quantized linear layers
first_layer = model.model.layers[0]
print(first_layer)

print(f"\n--- Memory comparison ---")
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")
print(f"Estimated fp16 memory: {total_params * 2 / 1e9:.2f} GB")
print(f"Estimated 4-bit memory: {total_params * 0.5 / 1e9:.2f} GB")
print(f"Actual GPU usage: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"Memory saved: ~{(total_params * 2 / 1e9) - (torch.cuda.memory_allocated() / 1e9):.1f} GB")

### Checkpoint 1 Summary

You have:
- Loaded a 7B model in 4-bit NF4 quantization
- Verified GPU memory usage is roughly 3.5-4.5 GB (vs ~14 GB in fp16)
- Understood why NF4 is better than FP4 for transformer weights
- Seen that double quantization provides additional memory savings

You can pause here and resume at Checkpoint 2.

---

## Checkpoint 2: LoRA Configuration

### What is LoRA?

Full fine-tuning updates every parameter in the model. For a 7B model, that
means storing optimizer states for 7 billion parameters -- roughly 56 GB of
memory with Adam (8 bytes per parameter for momentum and variance).

**LoRA (Low-Rank Adaptation)** takes a different approach. Instead of updating
the full weight matrix W (shape d x d), LoRA freezes W and adds two small
matrices:

```
W' = W + B @ A
```

where A has shape (d, r) and B has shape (r, d), with r << d (e.g., r=16
vs d=4096). The product B @ A is a low-rank approximation of the weight update.

Only A and B are trainable. For r=16 and d=4096:
- Full weight matrix: 4096 x 4096 = 16.7M parameters
- LoRA matrices: (4096 x 16) + (16 x 4096) = 131K parameters
- Reduction: **128x fewer** trainable parameters per layer

### Key hyperparameters

- **r (rank)**: The inner dimension of the A and B matrices. Higher rank
  means more capacity but more parameters. Typical values: 8, 16, 32, 64.

- **lora_alpha**: A scaling factor applied to the LoRA update. The effective
  learning rate for LoRA is scaled by (alpha / r). Common practice: set
  alpha = 2 * r.

- **target_modules**: Which layers to add adapters to. Attention projections
  (q_proj, k_proj, v_proj, o_proj) are the standard choice. Adding MLP layers
  (gate_proj, up_proj, down_proj) increases capacity but also parameter count.

- **lora_dropout**: Dropout applied to the LoRA path. A small value (0.05)
  helps prevent overfitting on small datasets.

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Prepare the quantized model for training.
# This handles gradient checkpointing and layer norm casting.
model = prepare_model_for_kbit_training(model)

print("Model prepared for k-bit training.")
print_gpu_memory("After prepare_model_for_kbit_training")

In [ ]:
# Configure LoRA adapters
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Print the trainable parameter summary
model.print_trainable_parameters()

In [ ]:
# Detailed breakdown: trainable vs frozen parameters by module
trainable_params = 0
frozen_params = 0
trainable_by_module = {}

for name, param in model.named_parameters():
    num_params = param.numel()
    if param.requires_grad:
        trainable_params += num_params
        # Group by module type
        module_type = name.split(".")[-2] if "." in name else name
        trainable_by_module[module_type] = (
            trainable_by_module.get(module_type, 0) + num_params
        )
    else:
        frozen_params += num_params

total_params = trainable_params + frozen_params
pct_trainable = 100 * trainable_params / total_params

print("Parameter Summary")
print("=" * 50)
print(f"  Frozen parameters:    {frozen_params:>14,}")
print(f"  Trainable parameters: {trainable_params:>14,}")
print(f"  Total parameters:     {total_params:>14,}")
print(f"  Trainable:            {pct_trainable:>13.4f}%")
print()
print("Trainable parameters by module:")
for module, count in sorted(trainable_by_module.items(), key=lambda x: -x[1]):
    print(f"  {module:<20} {count:>10,} ({100 * count / trainable_params:.1f}%)")

print(f"\n--- Memory comparison for optimizer ---")
print(f"Full fine-tuning (Adam, 8 bytes/param): {total_params * 8 / 1e9:.1f} GB")
print(f"LoRA fine-tuning (Adam, 8 bytes/param): {trainable_params * 8 / 1e9:.3f} GB")
print(f"Optimizer memory reduction: {total_params * 8 / (trainable_params * 8):.0f}x")

In [ ]:
# Visualize the parameter split
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: total vs trainable
ax = axes[0]
labels = ["Frozen (base model)", "Trainable (LoRA)"]
sizes = [frozen_params, trainable_params]
colors = ["#95a5a6", "#e74c3c"]
ax.pie(
    sizes, labels=labels, colors=colors, autopct="%1.2f%%",
    startangle=90, textprops={"fontsize": 11},
)
ax.set_title("Parameter Distribution", fontsize=13, fontweight="bold")

# Right: trainable parameter breakdown by module
ax = axes[1]
module_names = list(trainable_by_module.keys())
module_counts = list(trainable_by_module.values())
bar_colors = plt.cm.Set2(range(len(module_names)))
ax.barh(module_names, module_counts, color=bar_colors)
ax.set_xlabel("Number of Parameters")
ax.set_title("Trainable Parameters by Module", fontsize=13, fontweight="bold")
for i, count in enumerate(module_counts):
    ax.text(count + max(module_counts) * 0.01, i, f"{count:,}", va="center", fontsize=9)

plt.tight_layout()
plt.show()

print(f"LoRA adds only {pct_trainable:.4f}% trainable parameters.")
print(f"This is what makes fine-tuning a 7B model feasible on a single GPU.")

### Checkpoint 2 Summary

You have:
- Applied LoRA adapters to the attention layers (q, k, v, o projections)
- Verified that trainable parameters are less than 1% of the total
- Understood how rank, alpha, and target modules affect adapter size
- Seen the optimizer memory savings: LoRA requires orders of magnitude less

You can pause here and resume at Checkpoint 3.

---

## Checkpoint 3: Training

We use `trl.SFTTrainer` to fine-tune the model on legal instruction data.
SFTTrainer handles the details of supervised fine-tuning: chat template
formatting, loss masking (training only on assistant responses), and
integration with the PEFT/LoRA setup.

### Dataset

We load the instruction dataset built in Module 06 (or create one from
sample data if the Module 06 output is not available). The dataset
contains instruction-response pairs derived from court opinions:
summarizations, holding extractions, citation identification, and more.

### Training configuration

Key hyperparameters:
- **batch_size=4, gradient_accumulation=4**: effective batch size of 16
- **learning_rate=2e-4**: standard for LoRA training (higher than full fine-tuning)
- **cosine scheduler with warmup**: gradual warmup prevents early instability
- **bf16=True**: mixed precision for faster training on Ampere+ GPUs
- **max_seq_length=512**: keeps memory manageable; increase if your data is longer

In [ ]:
from datasets import Dataset

# Try to load the dataset from Module 06 first
sft_dataset_path = Path("../06-sft/sft_dataset.json")
sample_data_path = Path("../../datasets/sample/court_opinions.jsonl")


def build_dataset_from_opinions(opinions_path):
    """Build instruction pairs from court opinions (same approach as Module 06)."""
    opinions = []
    with open(opinions_path) as f:
        for line in f:
            opinions.append(json.loads(line))

    dataset_rows = []
    for op in opinions:
        text = op["text"]
        sentences = [s.strip() for s in text.split(".") if s.strip()]

        # Summarize
        summary = ". ".join(sentences[:2]) + "." if len(sentences) >= 2 else text[:300]
        dataset_rows.append({
            "instruction": "Summarize this court opinion.",
            "input": text,
            "output": summary,
        })

        # Holding
        paragraphs = text.split("\n\n")
        holding = paragraphs[-1].strip() if paragraphs else text[-300:]
        dataset_rows.append({
            "instruction": "What was the holding in this case?",
            "input": text,
            "output": holding,
        })

        # Citations
        citation_list = "\n".join(f"- {c}" for c in op.get("citations", []))
        dataset_rows.append({
            "instruction": "List the key legal citations in this opinion.",
            "input": text,
            "output": citation_list,
        })

        # Court
        dataset_rows.append({
            "instruction": "What court issued this opinion?",
            "input": text,
            "output": op["court"],
        })

        # Key issues
        issue_summary = (
            f"In {op['case_name']}, the key legal issue is: {sentences[0]}."
            if sentences
            else f"The key issue in {op['case_name']} concerns the matters "
                 f"described in the opinion."
        )
        dataset_rows.append({
            "instruction": "What are the key legal issues in this case?",
            "input": text,
            "output": issue_summary,
        })

    return dataset_rows


# Load or build the dataset
if sft_dataset_path.exists():
    with open(sft_dataset_path) as f:
        raw_dataset = json.load(f)
    print(f"Loaded {len(raw_dataset)} examples from Module 06: {sft_dataset_path}")
elif sample_data_path.exists():
    raw_dataset = build_dataset_from_opinions(sample_data_path)
    print(f"Built {len(raw_dataset)} examples from sample court opinions")
else:
    raise FileNotFoundError(
        "No dataset found. Please run Module 06 first or ensure "
        "datasets/sample/court_opinions.jsonl exists."
    )

print(f"\nExample:")
print(f"  Instruction: {raw_dataset[0]['instruction']}")
print(f"  Input: {raw_dataset[0]['input'][:100]}...")
print(f"  Output: {raw_dataset[0]['output'][:100]}...")

In [ ]:
# Format dataset as chat conversations for SFTTrainer
SYSTEM_PROMPT = (
    "You are a legal research assistant. Answer the question about the "
    "provided court opinion accurately and concisely."
)


def format_as_chat(example):
    """Convert an instruction example into a chat-format conversation."""
    user_content = example["instruction"]
    if example.get("input"):
        user_content += "\n\n" + example["input"]

    return {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": example["output"]},
        ]
    }


chat_dataset = [format_as_chat(ex) for ex in raw_dataset]
train_dataset = Dataset.from_list(chat_dataset)

print(f"Training dataset: {len(train_dataset)} examples")
print(f"\nFirst example messages:")
for msg in train_dataset[0]["messages"]:
    content_preview = msg["content"][:80] + "..." if len(msg["content"]) > 80 else msg["content"]
    print(f"  [{msg['role']}]: {content_preview}")

In [ ]:
from trl import SFTTrainer, SFTConfig

output_dir = "./qlora-legal"

training_args = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    max_seq_length=512,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    report_to="none",
)

print("Training configuration:")
print(f"  Epochs:             {training_args.num_train_epochs}")
print(f"  Batch size:         {training_args.per_device_train_batch_size}")
print(f"  Gradient accum:     {training_args.gradient_accumulation_steps}")
print(f"  Effective batch:    {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate:      {training_args.learning_rate}")
print(f"  Scheduler:          {training_args.lr_scheduler_type}")
print(f"  Warmup ratio:       {training_args.warmup_ratio}")
print(f"  Max seq length:     {training_args.max_seq_length}")
print(f"  Optimizer:          {training_args.optim}")
print(f"  Output dir:         {output_dir}")

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    processing_class=tokenizer,
)

print(f"Trainer created.")
print(f"Training steps: {trainer.state.max_steps if hasattr(trainer.state, 'max_steps') else 'will be computed at train time'}")
print_gpu_memory("Before training")

In [ ]:
# Train the model
print("Starting training...")
print("Monitor GPU memory in another terminal with: nvidia-smi -l 5")
print()

train_result = trainer.train()

print(f"\nTraining complete!")
print(f"  Total steps: {train_result.global_step}")
print(f"  Training loss: {train_result.training_loss:.4f}")
print_gpu_memory("After training")

In [ ]:
# Plot the training loss curve
log_history = trainer.state.log_history

# Extract training loss entries (filter out eval/save entries)
train_losses = [
    (entry["step"], entry["loss"])
    for entry in log_history
    if "loss" in entry
]

if train_losses:
    steps, losses = zip(*train_losses)

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(steps, losses, color="steelblue", linewidth=2, marker="o", markersize=4)
    ax.set_xlabel("Training Step", fontsize=12)
    ax.set_ylabel("Loss", fontsize=12)
    ax.set_title("QLoRA Training Loss", fontsize=14, fontweight="bold")
    ax.grid(True, alpha=0.3)

    # Mark epoch boundaries
    total_steps = steps[-1]
    steps_per_epoch = total_steps / training_args.num_train_epochs
    for epoch in range(1, int(training_args.num_train_epochs) + 1):
        epoch_step = int(epoch * steps_per_epoch)
        ax.axvline(x=epoch_step, color="red", linestyle="--", alpha=0.5)
        ax.text(epoch_step, max(losses) * 0.95, f"Epoch {epoch}",
                ha="center", fontsize=9, color="red")

    plt.tight_layout()
    plt.show()

    print(f"Initial loss: {losses[0]:.4f}")
    print(f"Final loss:   {losses[-1]:.4f}")
    print(f"Reduction:    {(1 - losses[-1] / losses[0]) * 100:.1f}%")
else:
    print("No training loss entries found in log history.")

In [ ]:
# Save the LoRA adapter weights
adapter_path = os.path.join(output_dir, "final-adapter")
model.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path)

# Check adapter size on disk
adapter_size_bytes = sum(
    f.stat().st_size for f in Path(adapter_path).rglob("*") if f.is_file()
)
adapter_size_mb = adapter_size_bytes / 1e6

print(f"Adapter saved to: {adapter_path}")
print(f"Adapter size on disk: {adapter_size_mb:.1f} MB")
print(f"\nFor comparison, the full model would be ~14 GB on disk.")
print(f"The adapter is {14000 / adapter_size_mb:.0f}x smaller.")
print(f"\nFiles saved:")
for f in sorted(Path(adapter_path).iterdir()):
    print(f"  {f.name} ({f.stat().st_size / 1e6:.1f} MB)")

### Checkpoint 3 Summary

You have:
- Loaded and formatted a legal instruction dataset
- Configured SFTTrainer with LoRA-appropriate hyperparameters
- Trained the QLoRA model and observed the loss curve
- Saved the adapter weights (much smaller than the full model)

You can pause here and resume at Checkpoint 4.

---

## Checkpoint 4: Evaluation and Merging

Now we evaluate the fine-tuned model by comparing its outputs to the base
model on legal prompts. Then we merge the adapters back into the base
model weights, producing a single standalone model.

### Base model vs QLoRA comparison

The base model (Mistral-7B-v0.1 or Llama-2-7B) was trained on general
web text. It knows about law in a general sense, but it was not trained
to follow instructions or produce structured legal analysis. After QLoRA
fine-tuning on our legal dataset, the model should:

1. Follow the instruction format (answer what was asked)
2. Produce more relevant, focused legal responses
3. Reference appropriate legal concepts from the training data

In [ ]:
# Define legal evaluation prompts
eval_prompts = [
    {
        "instruction": "Summarize this court opinion.",
        "input": (
            "The plaintiff filed suit alleging violations of the Americans "
            "with Disabilities Act after being terminated from his position "
            "as a warehouse supervisor. The district court granted summary "
            "judgment for the defendant employer, finding that the plaintiff "
            "failed to establish that he was qualified to perform the "
            "essential functions of the job with or without reasonable "
            "accommodation. The Seventh Circuit reviews this de novo."
        ),
    },
    {
        "instruction": "What was the holding in this case?",
        "input": (
            "The court considered whether the SEC had authority to revoke "
            "the registration of an investment adviser who systematically "
            "allocated IPO shares to proprietary accounts ahead of client "
            "orders. The Commission found willful violations of the "
            "Investment Advisers Act and imposed revocation."
        ),
    },
    {
        "instruction": "What are the key legal issues in this case?",
        "input": (
            "Parents of a child with autism challenged the school district's "
            "individualized education program under the Individuals with "
            "Disabilities Education Act. They argued that the district "
            "failed to consider independent educational evaluations and "
            "denied their child a free appropriate public education."
        ),
    },
]

print(f"Prepared {len(eval_prompts)} evaluation prompts")
for i, prompt in enumerate(eval_prompts):
    print(f"  [{i}] {prompt['instruction']}")

In [ ]:
def generate_response(model, tokenizer, instruction, input_text, max_new_tokens=256):
    """Generate a response from the model using a chat-formatted prompt."""
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"{instruction}\n\n{input_text}"},
    ]

    # Apply the chat template
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )

    # Decode only the generated tokens (skip the prompt)
    generated = outputs[0][inputs["input_ids"].shape[1]:]
    return tokenizer.decode(generated, skip_special_tokens=True)


# Generate responses with the QLoRA fine-tuned model
print("Generating responses with the QLoRA fine-tuned model...")
print("=" * 70)

qlora_responses = []
for i, prompt in enumerate(eval_prompts):
    response = generate_response(
        model, tokenizer, prompt["instruction"], prompt["input"]
    )
    qlora_responses.append(response)
    print(f"\n--- Prompt {i}: {prompt['instruction']} ---")
    print(f"QLoRA response:\n{response}")
    print()

In [ ]:
# Generate responses with the base model (disable LoRA adapters)
print("Generating responses with the BASE model (LoRA disabled)...")
print("=" * 70)

model.disable_adapter_layers()

base_responses = []
for i, prompt in enumerate(eval_prompts):
    response = generate_response(
        model, tokenizer, prompt["instruction"], prompt["input"]
    )
    base_responses.append(response)
    print(f"\n--- Prompt {i}: {prompt['instruction']} ---")
    print(f"Base response:\n{response}")
    print()

# Re-enable adapters
model.enable_adapter_layers()

In [ ]:
# Side-by-side comparison
print("Side-by-Side Comparison: Base Model vs QLoRA Fine-Tuned")
print("=" * 80)

for i, prompt in enumerate(eval_prompts):
    print(f"\n{'=' * 80}")
    print(f"Prompt {i}: {prompt['instruction']}")
    print(f"Input: {prompt['input'][:100]}...")
    print(f"{'=' * 80}")
    print(f"\n  BASE MODEL:")
    print(f"  {base_responses[i][:300]}")
    print(f"\n  QLORA FINE-TUNED:")
    print(f"  {qlora_responses[i][:300]}")
    print()

print("\nObservations:")
print("- The base model may produce continuations rather than direct answers.")
print("- The QLoRA model should follow the instruction format more closely.")
print("- Quality depends heavily on dataset size and training epochs.")

In [ ]:
# Simplified evaluation harness (adapted from Module 08)
import re


def response_relevance_score(instruction, response):
    """Score how relevant the response is to the instruction.

    Heuristic: measures overlap between instruction keywords and response.
    """
    stopwords = {
        "the", "a", "an", "is", "are", "was", "were", "be", "been",
        "have", "has", "had", "do", "does", "did", "will", "would",
        "to", "of", "in", "for", "on", "with", "at", "by", "from",
        "and", "but", "or", "not", "this", "that", "it", "what",
    }
    inst_words = set(re.findall(r"\b\w+\b", instruction.lower())) - stopwords
    resp_words = set(re.findall(r"\b\w+\b", response.lower())) - stopwords

    if not inst_words:
        return 0.0
    return len(inst_words & resp_words) / len(inst_words)


def response_length_score(response, target_min=50, target_max=500):
    """Score whether the response length is in a reasonable range."""
    length = len(response.split())
    if length < target_min:
        return length / target_min
    elif length > target_max:
        return max(0.5, target_max / length)
    return 1.0


# Score both model outputs
print("Simplified Evaluation Scores")
print("=" * 70)
print(f"{'Prompt':>8}  {'Model':<12}  {'Relevance':>10}  {'Length':>8}  {'Combined':>9}")
print("-" * 70)

for i, prompt in enumerate(eval_prompts):
    for label, response in [("Base", base_responses[i]), ("QLoRA", qlora_responses[i])]:
        relevance = response_relevance_score(prompt["instruction"], response)
        length = response_length_score(response)
        combined = 0.6 * relevance + 0.4 * length
        print(f"  {i:>5}  {label:<12}  {relevance:>10.3f}  {length:>8.3f}  {combined:>9.3f}")

print()
print("Note: These heuristic scores give a rough signal. For rigorous")
print("evaluation, use the full harness from Module 08.")

### Merging Adapters

LoRA adapters can be used in two ways:

1. **Keep adapters separate**: Load the base model, then load adapters on top.
   Advantages: you can swap adapters (e.g., one for legal, one for medical),
   share the base model across tasks, and the adapter files are tiny.

2. **Merge adapters into the base model**: Combine W + BA into a single weight
   matrix. Advantages: simpler deployment (one model file), slightly faster
   inference (no adapter overhead), compatible with any serving framework.

When to keep separate:
- You have multiple domain adapters for one base model
- You want to A/B test different adapter versions
- Storage is a concern (adapters are ~50 MB vs ~14 GB merged)

When to merge:
- You are deploying a single model for production
- You need maximum inference speed
- You want to quantize the merged model further (e.g., GGUF for llama.cpp)

In [ ]:
# Merge LoRA adapters back into the base model
print("Merging LoRA adapters into base model...")
print_gpu_memory("Before merge")

merged_model = model.merge_and_unload()

print_gpu_memory("After merge")
print(f"\nMerged model type: {type(merged_model).__name__}")
print(f"The model is now a standard transformers model with no adapter layers.")

In [ ]:
# Compare adapter-only vs merged model size
merged_path = os.path.join(output_dir, "merged-model")
merged_model.save_pretrained(merged_path)
tokenizer.save_pretrained(merged_path)

merged_size_bytes = sum(
    f.stat().st_size for f in Path(merged_path).rglob("*") if f.is_file()
)
merged_size_gb = merged_size_bytes / 1e9

print("Size Comparison")
print("=" * 40)
print(f"  Adapter only:  {adapter_size_mb:>8.1f} MB")
print(f"  Merged model:  {merged_size_gb:>8.2f} GB")
print(f"  Ratio:         {merged_size_gb * 1000 / adapter_size_mb:>8.0f}x")
print()
print("The adapter is tiny because it only stores the LoRA matrices.")
print("The merged model includes all base model weights with adapters baked in.")

In [ ]:
# Verify the merged model produces the same outputs
print("Verifying merged model output matches QLoRA adapter output...")

test_prompt = eval_prompts[0]
merged_response = generate_response(
    merged_model, tokenizer,
    test_prompt["instruction"], test_prompt["input"],
    max_new_tokens=100,
)

print(f"\nMerged model response:")
print(f"  {merged_response[:300]}")
print()
print("Note: Due to sampling (temperature > 0), outputs may differ between")
print("runs. Set temperature=0 for deterministic comparison.")

### Checkpoint 4 Summary

You have:
- Generated and compared outputs from the base model vs QLoRA fine-tuned model
- Run a simplified evaluation harness to score responses
- Merged LoRA adapters back into the base model
- Compared adapter-only size (~50 MB) vs merged model size (~14 GB)
- Understood when to keep adapters separate vs merge

You can pause here and resume at Checkpoint 5.

---

## Checkpoint 5: Experiments

This checkpoint provides a framework for experimenting with QLoRA
hyperparameters. The goal is to understand how different choices affect
model quality and training cost.

Each experiment modifies one variable while keeping everything else
constant. Record your results in the comparison table at the end.

In [ ]:
def run_qlora_experiment(
    model_name,
    lora_r,
    lora_alpha,
    target_modules,
    num_epochs,
    train_dataset,
    tokenizer,
    experiment_name,
):
    """Run a complete QLoRA training experiment with given hyperparameters.

    Args:
        model_name: HuggingFace model ID.
        lora_r: LoRA rank.
        lora_alpha: LoRA alpha scaling factor.
        target_modules: List of module names to apply LoRA to.
        num_epochs: Number of training epochs.
        train_dataset: HuggingFace Dataset with 'messages' column.
        tokenizer: Tokenizer instance.
        experiment_name: Name for the output directory.

    Returns:
        Dict with experiment results.
    """
    from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
    from trl import SFTTrainer, SFTConfig

    print(f"\n{'=' * 60}")
    print(f"Experiment: {experiment_name}")
    print(f"  r={lora_r}, alpha={lora_alpha}, targets={target_modules}, epochs={num_epochs}")
    print(f"{'=' * 60}")

    # Load fresh quantized model
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

    exp_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.bfloat16,
    )
    exp_model = prepare_model_for_kbit_training(exp_model)

    # Apply LoRA
    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=target_modules,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )
    exp_model = get_peft_model(exp_model, lora_config)

    trainable = sum(p.numel() for p in exp_model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in exp_model.parameters())
    print(f"  Trainable: {trainable:,} / {total:,} ({100 * trainable / total:.4f}%)")

    # Training
    exp_output_dir = f"./qlora-experiments/{experiment_name}"
    training_args = SFTConfig(
        output_dir=exp_output_dir,
        num_train_epochs=num_epochs,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        weight_decay=0.01,
        warmup_ratio=0.03,
        lr_scheduler_type="cosine",
        logging_steps=5,
        save_strategy="no",
        bf16=True,
        max_seq_length=512,
        gradient_checkpointing=True,
        optim="paged_adamw_8bit",
        report_to="none",
    )

    trainer = SFTTrainer(
        model=exp_model,
        args=training_args,
        train_dataset=train_dataset,
        processing_class=tokenizer,
    )

    train_result = trainer.train()

    # Extract loss curve
    losses = [
        (entry["step"], entry["loss"])
        for entry in trainer.state.log_history
        if "loss" in entry
    ]

    final_loss = losses[-1][1] if losses else float("nan")
    initial_loss = losses[0][1] if losses else float("nan")

    result = {
        "name": experiment_name,
        "r": lora_r,
        "alpha": lora_alpha,
        "target_modules": target_modules,
        "num_epochs": num_epochs,
        "trainable_params": trainable,
        "initial_loss": initial_loss,
        "final_loss": final_loss,
        "loss_curve": losses,
        "total_steps": train_result.global_step,
        "model": exp_model,
    }

    print(f"  Final loss: {final_loss:.4f}")
    print(f"  Steps: {train_result.global_step}")

    return result


print("Experiment runner defined.")
print("Run the cells below to execute experiments.")
print("Each experiment loads a fresh model, so they are independent.")

### Experiment 1: LoRA Rank

How does the rank (r) affect model quality? Lower rank means fewer
parameters but less capacity to learn. Higher rank means more parameters
and potentially better quality, but diminishing returns.

We test r=4, r=16, and r=64.

In [ ]:
# Experiment 1: Vary LoRA rank
# NOTE: This cell runs 3 full training runs. It may take a while.
# You can reduce num_epochs to 1 for faster iteration.

rank_experiments = []
attention_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

for r in [4, 16, 64]:
    result = run_qlora_experiment(
        model_name=model_name,
        lora_r=r,
        lora_alpha=2 * r,  # Keep alpha = 2 * r as convention
        target_modules=attention_modules,
        num_epochs=3,
        train_dataset=train_dataset,
        tokenizer=tokenizer,
        experiment_name=f"rank-{r}",
    )
    rank_experiments.append(result)

    # Free GPU memory between experiments
    del result["model"]
    torch.cuda.empty_cache()

print("\nAll rank experiments complete.")

In [ ]:
# Plot rank experiment results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: loss curves
ax = axes[0]
colors = ["#e74c3c", "#3498db", "#2ecc71"]
for exp, color in zip(rank_experiments, colors):
    if exp["loss_curve"]:
        steps, losses = zip(*exp["loss_curve"])
        ax.plot(steps, losses, label=f"r={exp['r']}", color=color, linewidth=2)
ax.set_xlabel("Step")
ax.set_ylabel("Loss")
ax.set_title("Training Loss by LoRA Rank")
ax.legend()
ax.grid(True, alpha=0.3)

# Right: final loss vs trainable params
ax = axes[1]
ranks = [exp["r"] for exp in rank_experiments]
final_losses = [exp["final_loss"] for exp in rank_experiments]
param_counts = [exp["trainable_params"] / 1e6 for exp in rank_experiments]

ax.bar(range(len(ranks)), final_losses, color=colors, alpha=0.8)
ax.set_xticks(range(len(ranks)))
ax.set_xticklabels([f"r={r}\n({p:.1f}M params)" for r, p in zip(ranks, param_counts)])
ax.set_ylabel("Final Loss")
ax.set_title("Final Loss by LoRA Rank")
ax.grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.show()

print("Results:")
for exp in rank_experiments:
    print(f"  r={exp['r']:>3}: {exp['trainable_params']:>10,} params, "
          f"final_loss={exp['final_loss']:.4f}")

### Experiment 2: Target Modules

By default, LoRA is applied to attention projections only. Adding MLP
layers (gate_proj, up_proj, down_proj) increases adapter capacity but
also parameter count. Does the extra capacity help?

In [ ]:
# Experiment 2: Attention only vs Attention + MLP
module_configs = {
    "attention-only": ["q_proj", "k_proj", "v_proj", "o_proj"],
    "attention-mlp": [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
}

module_experiments = []

for name, modules in module_configs.items():
    result = run_qlora_experiment(
        model_name=model_name,
        lora_r=16,
        lora_alpha=32,
        target_modules=modules,
        num_epochs=3,
        train_dataset=train_dataset,
        tokenizer=tokenizer,
        experiment_name=name,
    )
    module_experiments.append(result)

    del result["model"]
    torch.cuda.empty_cache()

print("\nModule experiments complete.")
for exp in module_experiments:
    print(f"  {exp['name']}: {exp['trainable_params']:,} params, "
          f"final_loss={exp['final_loss']:.4f}")

### Experiment 3: Number of Epochs

With a small dataset, training for more epochs risks overfitting.
We test 1, 3, and 5 epochs to find the sweet spot.

In [ ]:
# Experiment 3: Vary number of epochs
epoch_experiments = []

for n_epochs in [1, 3, 5]:
    result = run_qlora_experiment(
        model_name=model_name,
        lora_r=16,
        lora_alpha=32,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        num_epochs=n_epochs,
        train_dataset=train_dataset,
        tokenizer=tokenizer,
        experiment_name=f"epochs-{n_epochs}",
    )
    epoch_experiments.append(result)

    del result["model"]
    torch.cuda.empty_cache()

print("\nEpoch experiments complete.")
for exp in epoch_experiments:
    print(f"  {exp['num_epochs']} epochs: final_loss={exp['final_loss']:.4f}, "
          f"steps={exp['total_steps']}")

In [ ]:
# Compile all experiment results into a comparison table
all_experiments = rank_experiments + module_experiments + epoch_experiments

print("Experiment Results Comparison")
print("=" * 90)
print(
    f"{'Experiment':<20} {'Rank':>5} {'Modules':>15} {'Epochs':>7} "
    f"{'Params':>12} {'Init Loss':>10} {'Final Loss':>11}"
)
print("-" * 90)

for exp in all_experiments:
    module_str = "attn" if len(exp["target_modules"]) <= 4 else "attn+mlp"
    print(
        f"{exp['name']:<20} {exp['r']:>5} {module_str:>15} {exp['num_epochs']:>7} "
        f"{exp['trainable_params']:>12,} {exp['initial_loss']:>10.4f} {exp['final_loss']:>11.4f}"
    )

print("=" * 90)
print()
print("Use this table to identify the best configuration for your use case.")
print("Lower final loss generally indicates better training, but watch for")
print("overfitting (final loss much lower than initial with a small dataset).")

### Checkpoint 5 Summary

You have:
- Experimented with different LoRA ranks (4, 16, 64)
- Compared attention-only vs attention+MLP target modules
- Tested different numbers of training epochs (1, 3, 5)
- Recorded results in a comparison table

Key takeaways:
- Higher rank increases capacity but with diminishing returns
- Adding MLP modules increases parameters significantly but may improve quality
- More epochs help up to a point; then overfitting kicks in
- The optimal configuration depends on your dataset size and quality requirements

---

## Exercises

### Exercise (a): LoRA Rank Exploration

Experiment with LoRA rank values of 4, 16, and 64. For each rank:

1. Train a QLoRA model using `run_qlora_experiment()`.
2. Generate responses to the `eval_prompts` defined in Checkpoint 4.
3. Score each model using the simplified evaluation harness.
4. Record the results in a table: rank, trainable params, final loss,
   relevance score, and your subjective quality assessment.

Questions to consider:
- At what rank do you see diminishing returns?
- Is there a rank where the model begins to overfit on this small dataset?
- How does training time scale with rank?

```python
# Starter code for Exercise (a)
for r in [4, 16, 64]:
    result = run_qlora_experiment(
        model_name=model_name,
        lora_r=r,
        lora_alpha=2 * r,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        num_epochs=3,
        train_dataset=train_dataset,
        tokenizer=tokenizer,
        experiment_name=f"exercise-a-rank-{r}",
    )
    # Generate responses and score them here
```

### Exercise (b): Target Module Comparison

Compare two LoRA configurations:
1. **Attention only**: `["q_proj", "k_proj", "v_proj", "o_proj"]`
2. **Attention + MLP**: `["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]`

For each configuration:
1. Train with r=16, 3 epochs.
2. Compare trainable parameter counts.
3. Generate and compare responses on the same prompts.
4. Measure whether the extra parameters in the MLP configuration
   improve response quality enough to justify the increased cost.

```python
# Starter code for Exercise (b)
configs = {
    "attention": ["q_proj", "k_proj", "v_proj", "o_proj"],
    "attention_mlp": [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
}
for name, modules in configs.items():
    result = run_qlora_experiment(
        model_name=model_name,
        lora_r=16,
        lora_alpha=32,
        target_modules=modules,
        num_epochs=3,
        train_dataset=train_dataset,
        tokenizer=tokenizer,
        experiment_name=f"exercise-b-{name}",
    )
    # Compare outputs here
```

### Exercise (c): Larger Dataset

The training dataset in this lab is small (derived from 5 court opinions).
Try training on a larger dataset:

1. Use the scripts in `datasets/scripts/` to fetch more court opinions:
   ```bash
   python ../../datasets/scripts/fetch_courtlistener.py --limit 50
   ```
2. Build instruction pairs from the expanded dataset using the same
   approach as Module 06.
3. Train a QLoRA model on the larger dataset.
4. Compare outputs and loss curves with the small-dataset model.

Questions to consider:
- How does the loss curve change with more data?
- Do you need more or fewer epochs with a larger dataset?
- Is the quality improvement proportional to the data increase?

---

## Cleanup

The training outputs can consume significant disk space. Run the cell
below to remove experiment artifacts if you no longer need them.

In [ ]:
# Uncomment to clean up experiment outputs
# import shutil
# for dir_name in ["qlora-legal", "qlora-experiments"]:
#     if Path(dir_name).exists():
#         shutil.rmtree(dir_name)
#         print(f"Removed {dir_name}")

# Free GPU memory
# del model, merged_model
# torch.cuda.empty_cache()
# print_gpu_memory("After cleanup")